# 서포트 벡터 머신(SVM)
매우 강력하고 선형이나 비선형 분류, 회귀, 이상치 탐색에도 사용할 수 있는 다목적 머신러닝 모델.  
머신러닝에서 가장 인기있는 모델 중 하나. 관심있는 사람이라면 무조건 알아야 하는 모델.  
>**특히 복잡한 분류 문제에 대해 잘 들어맞음. 작거나 중간 크기의 데이터셋에 적합함**

SVM은 **라지 마진 분류** 라고도 함.  
결정 경계는 무조건 **서포트 벡터** 에 의해서만 결정됨. 즉 경계 바깥에 샘플이 아무리 추가되도 경계에 영향을 안 미침.  
> 모든 샘플이 경계 바깥쪽에 올바르게 분류되어 있다면 : **하드 마진 분류**  
이때 데이터가 선형적으로 구분될 수 있어야 제대로 작동함.  
그리고 이상치에 민감함.
  
> 하드 마진 분류보다 좀 더 유연한 **소프트 마진 분류**  
경계의 폭을 가능한 한 넓게 유지하는 것과 마진 오류(샘플이 도로 중간이나 반대쪽에 있는 경우) 사이에 균형을 잡아야 함.  

파라미터 **C** 가 그 역할을 함
> **C가 클수록** 모델 규제가 풀림 -> **과대적합화** -> 적은 마진으로 분류(마진 오류를 조금 허용함)  
**C가 작을수록** 모델 규제가 커짐 -> **과소적합화** -> 큰 마진으로 분류(마진 오류를 많이 허용함)  
  
> 논리 회귀처럼 각 클래스에 대한 확률을 직접 제공하진 않음.  
대신에 경계선으로부터 얼마나 멀리 떨어져 있는지에 대한 점수는 얻을 수 있음 -> 이를 확률처럼 활용은 가능

---
## 1. 선형 SVM 분류

In [6]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
x = iris["data"][:, (2, 3)]   # 꽃잎 길이, 꽃잎 너비만
y = (iris["target"] == 2).astype(np.int)     # 버지니카 데이터만

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

In [7]:
svm_clf.fit(x, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [8]:
svm_clf.predict([[5.5, 1.7]])

array([1])

LinearSVC대신 선형 커널을 사용하는 SVC클래스를 이용할 수도 있음. 그런데 속도가 LiearSVC가 훨씬 빠르다고 함.  
LinearSVC는 규제에 편향을 포함시킴 -> 따라서 학습데이터에서 평균을 빼 중앙에 맞춰줘야 함. (**스케일링 꼭 필요!**)

---
## 2. 비선형 SVM 분류
비선형 데이터를 다루는 방법은 이전처럼 다항 특성과같은 특성을 더 추가하는 것.  
하지만 낮은 차원의 다항식은 매우 복잡한 데이터를 표현하지 못하고, 높은 차원의 다항식은 너무 많은 특성을 추가함.  
  
SVM은 이를 **커널 트릭** 을 이용하여 해결함.  
커널 트릭은 실제로는 특성을 추가하지 않으나 많이 추가한 것과 같은 결과를 얻을 수 있다고 함.(??)  

In [10]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

**coef0** 은 커널이 "poly", "sigmoid"일 때만 사용됨.  
모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을지 조절한다고 함?  
> 물론 최적의 파라미터를 찾는 것은 **그리드 탐색**을 이용  
**처음엔 폭을 크게 하여** 빠르게 감을 찾고  
**대략적인 좋은 파라미터를 찾음녀** 그 다음에는 그리드를 세밀하게 검색

In [11]:
poly_kernel_svm_clf.fit(x, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

### 2.1 유사도 특성
비선형 특성을 다루는 또 다른 기법은  
>샘플이 특정 **랜드마크** 와 얼마나 닮았는지 측정하는 **유사도 함수** 로 계산한 특성을 추가하는 것  
여기선 **방사 기저 함수(radial basis function : RBF)** 를 유사도 함수로 정의  

파라미터로 **감마(gamma)** 가 사용됨. **C** 와 비슷한 역할을 함.  
> **감마가 클수록** 유사도 함수의 폭이 줄어듦 -> 결정 경계가 불규칙해지고 각 샘플을 따라 구불구불해짐 -> **과대적합화**  
**감마가 작을수록** 유사도 함수의 폭이 커짐 -> 결정 경계가 부드러워짐 -> **과소적합화**

### 2.2 가우시안 RBF 커널

In [12]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

### 2.3 커널 고르기
> 언제나 **선형 커널을 먼저** 시도해보기  
특히 학습 데이터셋이 아주 크거나 특성 수가 많을 경우! (다른 커널 방법은 데이터셋이 클수록 오래 걸려서)  
이때 SVC(kernel='linear')보다 LinearSVC가 훨씬 빠르다는 것을 염두  

> 훈련 세트가 너무 크지 않다면 **가우시안 RBF 커널** 시도해보기  
대부분의 경우 잘 들어맞음.  

> 시간과 컴퓨팅 성능이 충분하다면 **교차 검증 + 그리드 탐색** 을 이용해 다양한 커널 시도해보기

---
## 3. 계산 복잡도
SVC의 훈련 시간 복잡도는 O(m^2 x n) ~ O(m^3 x n) 사이라고 함.  
**즉 훈련 샘플 수가 커질수록 엄청나게 느려짐**  
> 그래서 **복잡하지만(특성 수가 많은)**, **작거나 중간 규모(샘플의 수가 많지 않은)**의 훈련 세트에 잘 맞음  
 특히 희소 특성인 경우에 잘 확장됨.

---
## 4. SVM 회귀
svm을 분류에 적용할 경우  
> 일정한 마진 오류 내에서 두 클래스 간의 폭이 가능한 한 최대가 되도록 학습  
  
svm을 회귀에 적용할 경우  
> 제한된 마진 오류 내에서 가능한 한 많은 샘플이 들어가도록 학습

도로의 폭은 **엡실론(epsilon)** 으로 조절  
> **엡실론이 커질수록** 마진 오류(도로폭)를 크게  
**엡실론이 작을수록** 마진 오류를 작게
  
> 이때 마진 내에서는 훈련 샘플이 추가되더라고 모델의 예측에는 전혀 영향이 없음.  
그래서 이 모델을 **epsilon-insensitive(엡실론에 민감하지 않다)** 라고 함.

In [13]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)

In [14]:
svm_reg.fit(x, y)

LinearSVR(epsilon=1.5)

단순 선형 예측이 아닌 비선형 회귀 작업을 처리하려면 **커널 SVM** 모델을 사용.

In [15]:
from sklearn.svm import SVR

poly_kernel_svm_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)

In [16]:
poly_kernel_svm_reg.fit(x, y)

SVR(C=100, degree=2, kernel='poly')

LinearSVR, LinearSVC같이 Linear가 붙으면 학습 시간이 샘플 수에 비례해서 선형적으로 늘어난다고 함.  
일반 SVR, SVC는 훨씬 느림. 그래서 Linear를 먼저 써보라는 듯